In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tqdm import tqdm

TOKENIZER_PATH = "saved_tokenizer"
MODEL_PATH = "saved_model"
BATCH_SIZE = 16
LOCAL_DATA_FILE = "FinancialPhraseBank-v1.0/Sentences_AllAgree.txt" 

label_map = {
    "positive": 0,
    "negative": 1,
    "neutral": 2
}

reverse_label_map = {v: k for k, v in label_map.items()}


def evaluate_model():
    """
    Betölt egy lokálisan mentett modellt és tokenizert,
    hogy kiértékelje egy kézileg beolvasott txt adathalmazon.
    """
    
    print("--- 1. Modell és Tokenizer betöltése ---")
    try:
        tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
        print("Modell és tokenizer sikeresen betöltve.")
    except Exception as e:
        print(f"Hiba a modell/tokenizer betöltése közben: {MODEL_PATH} és {TOKENIZER_PATH}.")
        print(f"Hiba: {e}")
        return

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print(f"Modell áthelyezve a következő eszközre: {device}")

    print(f"\n--- 2. Fájl kézi beolvasása: {LOCAL_DATA_FILE} ---")
    
    sentences = []
    labels = []
    
    try:
        with open(LOCAL_DATA_FILE, 'r', encoding='latin-1') as f:
            for line in f:
                line = line.strip() 
                if not line:
                    continue
                
                parts = line.split('@')
                
                if len(parts) == 2:
                    sentence_text = parts[0]
                    label_text = parts[1]
                    
                    if label_text in label_map:
                        sentences.append(sentence_text)
                        labels.append(label_map[label_text])
                    else:
                        print(f"Figyelmeztetés: Ismeretlen címkéjű sor kihagyása: {line}")
                
                else:
                    print(f"Figyelmeztetés: Hibás formátumú sor kihagyása: {line}")

        print(f"Fájl sikeresen beolvasva. Találatok száma: {len(sentences)} minta.")

    except FileNotFoundError:
        print(f"Hiba: A fájl nem található: {LOCAL_DATA_FILE}")
        return
    except Exception as e:
        print(f"Hiba a fájl olvasása vagy feldolgozása közben: {e}")
        return

    print("\n--- 3. Tokenizálás és adatok előkészítése ---")

    print("Minden mondat tokenizálása...")
    tokenized_inputs = tokenizer(
        sentences,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    label_tensors = torch.tensor(labels, dtype=torch.long)

    test_dataset = TensorDataset(
        tokenized_inputs['input_ids'],
        tokenized_inputs['attention_mask'],
        label_tensors
    )

    # shuffle=False kulcsfontosságú, hogy a sorrend megmaradjon!
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    print("DataLoader létrehozva.")

    print(f"\n--- 4. Kiértékelés futtatása (Batch méret: {BATCH_SIZE}) ---")
    
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels_batch = batch[2].to(device) 

            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels_batch.cpu().numpy())

    print("\n--- 5. Metrikák és Hibaelemzés ---")
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    print("\n" + "="*30)
    print(f"          KIÉRTÉKELÉS BEFEJEZVE")
    print("="*30)
    print(f"Összes minta: {len(all_labels)}")
    print(f"Helyes jóslatok: {np.sum(np.array(all_labels) == np.array(all_preds))}")
    print("---")
    print(f"Accuracy:  {accuracy * 100:.2f}%")
    print(f"Precision (Weighted): {precision:.4f}")
    print(f"Recall (Weighted):    {recall:.4f}")
    print(f"F1-score (Weighted):  {f1:.4f}")
    print("="*30)

    print("\n--- Tévesen Címkézett Mondatok Elemzése ---")
    
    wrongly_labeled = []

    for i in range(len(all_labels)):
        true_label = all_labels[i]
        pred_label = all_preds[i]
        
        if true_label != pred_label:
            wrongly_labeled.append({
                "sentence": sentences[i],
                "true_label": reverse_label_map[true_label], 
                "predicted_label": reverse_label_map[pred_label]
            })

    if not wrongly_labeled:
        print("Minden minta helyesen lett címkézve! Nincs hiba.")
    else:
        print(f"Összesen {len(wrongly_labeled)} téves jóslat. Első 5 mutatása:")
        
        # Kiíratjuk az első 5-öt (vagy kevesebbet, ha nincs annyi)
        for i, item in enumerate(wrongly_labeled[:5]):
            print(f"\n{i+1}. HIBA")
            print(f"  Mondat:   {item['sentence']}")
            print(f"  Valódi:   {item['true_label']}")
            print(f"  Jósolt: {item['predicted_label']}")
    
    print("="*30)

if __name__ == "__main__":
    evaluate_model()

--- 1. Modell és Tokenizer betöltése ---
Modell és tokenizer sikeresen betöltve.
Modell áthelyezve a következő eszközre: cpu

--- 2. Fájl kézi beolvasása: bert_model/FinancialPhraseBank-v1.0/Sentences_AllAgree.txt ---
Fájl sikeresen beolvasva. Találatok száma: 2264 minta.

--- 3. Tokenizálás és adatok előkészítése ---
Minden mondat tokenizálása...
DataLoader létrehozva.

--- 4. Kiértékelés futtatása (Batch méret: 16) ---


Evaluating: 100%|██████████| 142/142 [07:25<00:00,  3.14s/it]


--- 5. Metrikák és Hibaelemzés ---

          KIÉRTÉKELÉS BEFEJEZVE
Összes minta: 2264
Helyes jóslatok: 2240
---
Accuracy:  98.94%
Precision (Weighted): 0.9895
Recall (Weighted):    0.9894
F1-score (Weighted):  0.9894

--- Tévesen Címkézett Mondatok Elemzése ---
Összesen 24 téves jóslat. Első 5 mutatása:

1. HIBA
  Mondat:   Finnish flexible packaging manufacturer Suominen Corporation reports net sales of EUR 54.5 mn in the first quarter of 2008 , compared with EUR 54.3 mn a year earlier .
  Valódi:   positive
  Jósolt: negative

2. HIBA
  Mondat:   These moderate but significant changes resulted in a significant 24-32 % reduction in the estimated CVD risk .
  Valódi:   positive
  Jósolt: negative

3. HIBA
  Mondat:   Investors will continue being interested in the company 's share although it is not quite cheap , Affarsvarlden said .
  Valódi:   positive
  Jósolt: neutral

4. HIBA
  Mondat:   At the end of the review period , Nordic Aluminium 's order book stood at EUR 8.77 mn compar